<a href="https://colab.research.google.com/github/Diya-Amith/Data-Science-Project-Galaxy-Morphology-Classification-Using-Deep-Learning-and-Explainable-AI-/blob/main/new_codetry.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **GALAXY MORPHOLOGY CLASSIFICATION USING DEEP LEARNING AND EXPLAINABLE AI**

## **Import Libraries and Initial Loading**

In [1]:
!pip install imbalanced-learn


In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from PIL import Image
import tensorflow as tf
from google.colab import drive
import cv2
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from sklearn.utils.class_weight import compute_class_weight

from sklearn.preprocessing import LabelEncoder


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
train_images_path = "/content/drive/MyDrive/galaxy-zoo-the-galaxy-challenge/images_training_rev1/images_training_rev1"
test_images_path = "/content/drive/MyDrive/galaxy-zoo-the-galaxy-challenge/images_test_rev1/images_test_rev1"
labels_path = "/content/drive/MyDrive/galaxy-zoo-the-galaxy-challenge/training_solutions_rev1/training_solutions_rev1.csv"
Data_path = "/content/drive/MyDrive/galaxy-zoo-the-galaxy-challenge"


## **Preprocessing**

### **Labels**

In [4]:
# Load labels
labels_df = pd.read_csv(labels_path)


In [5]:
labels_df.head()

,GalaxyID,Class1.1,Class1.2,Class1.3,Class2.1,Class2.2,Class3.1,Class3.2,Class4.1,Class4.2,...,Class9.3,Class10.1,Class10.2,Class10.3,Class11.1,Class11.2,Class11.3,Class11.4,Class11.5,Class11.6
0,100008,0.383147,0.616853,0.000000,0.000000,0.616853,0.038452,0.578401,0.418398,0.198455,...,0.000000,0.279952,0.138445,0.000000,0.000000,0.092886,0.0,0.0,0.0,0.325512
1,100023,0.327001,0.663777,0.009222,0.031178,0.632599,0.467370,0.165229,0.591328,0.041271,...,0.018764,0.000000,0.131378,0.459950,0.000000,0.591328,0.0,0.0,0.0,0.000000
2,100053,0.765717,0.177352,0.056931,0.000000,0.177352,0.000000,0.177352,0.000000,0.177352,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000
3,100078,0.693377,0.238564,0.068059,0.000000,0.238564,0.109493,0.129071,0.189098,0.049466,...,0.000000,0.094549,0.000000,0.094549,0.189098,0.000000,0.0,0.0,0.0,0.000000
4,100090,0.933839,0.000000,0.066161,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000


In [6]:
labels_df.shape

(61578, 38)

In [7]:
labels_df['image_path'] = labels_df['GalaxyID'].astype(str) + '.jpg'
labels_df.head()

,GalaxyID,Class1.1,Class1.2,Class1.3,Class2.1,Class2.2,Class3.1,Class3.2,Class4.1,Class4.2,...,Class10.1,Class10.2,Class10.3,Class11.1,Class11.2,Class11.3,Class11.4,Class11.5,Class11.6,image_path
0,100008,0.383147,0.616853,0.000000,0.000000,0.616853,0.038452,0.578401,0.418398,0.198455,...,0.279952,0.138445,0.000000,0.000000,0.092886,0.0,0.0,0.0,0.325512,100008.jpg
1,100023,0.327001,0.663777,0.009222,0.031178,0.632599,0.467370,0.165229,0.591328,0.041271,...,0.000000,0.131378,0.459950,0.000000,0.591328,0.0,0.0,0.0,0.000000,100023.jpg
2,100053,0.765717,0.177352,0.056931,0.000000,0.177352,0.000000,0.177352,0.000000,0.177352,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,100053.jpg
3,100078,0.693377,0.238564,0.068059,0.000000,0.238564,0.109493,0.129071,0.189098,0.049466,...,0.094549,0.000000,0.094549,0.189098,0.000000,0.0,0.0,0.0,0.000000,100078.jpg
4,100090,0.933839,0.000000,0.066161,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,100090.jpg


In [8]:
labels_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61578 entries, 0 to 61577
Data columns (total 39 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   GalaxyID    61578 non-null  int64  
 1   Class1.1    61578 non-null  float64
 2   Class1.2    61578 non-null  float64
 3   Class1.3    61578 non-null  float64
 4   Class2.1    61578 non-null  float64
 5   Class2.2    61578 non-null  float64
 6   Class3.1    61578 non-null  float64
 7   Class3.2    61578 non-null  float64
 8   Class4.1    61578 non-null  float64
 9   Class4.2    61578 non-null  float64
 10  Class5.1    61578 non-null  float64
 11  Class5.2    61578 non-null  float64
 12  Class5.3    61578 non-null  float64
 13  Class5.4    61578 non-null  float64
 14  Class6.1    61578 non-null  float64
 15  Class6.2    61578 non-null  float64
 16  Class7.1    61578 non-null  float64
 17  Class7.2    61578 non-null  float64
 18  Class7.3    61578 non-null  float64
 19  Class8.1    61578 non-nul

In [9]:
# missing values
missing_values = labels_df.isnull().sum()

print("Missing Values in Each Column:")
print(missing_values)

if missing_values.sum() > 0:
    print("\nThere are missing values in the dataset.")
else:
    print("\nNo missing values in the dataset.")

Missing Values in Each Column:
GalaxyID      0
Class1.1      0
Class1.2      0
Class1.3      0
Class2.1      0
Class2.2      0
Class3.1      0
Class3.2      0
Class4.1      0
Class4.2      0
Class5.1      0
Class5.2      0
Class5.3      0
Class5.4      0
Class6.1      0
Class6.2      0
Class7.1      0
Class7.2      0
Class7.3      0
Class8.1      0
Class8.2      0
Class8.3      0
Class8.4      0
Class8.5      0
Class8.6      0
Class8.7      0
Class9.1      0
Class9.2      0
Class9.3      0
Class10.1     0
Class10.2     0
Class10.3     0
Class11.1     0
Class11.2     0
Class11.3     0
Class11.4     0
Class11.5     0
Class11.6     0
image_path    0
dtype: int64

No missing values in the dataset.


### **Images**

In [10]:
# number of images
train_image_count = len(os.listdir(train_images_path))
test_image_count = len(os.listdir(test_images_path))


print(f"Number of images in training folder: {train_image_count}")
print(f"Number of images in testing folder: {test_image_count}")

Number of images in training folder: 17499
Number of images in testing folder: 26714


In [11]:
# list of image file names in the training folder
training_images = set(os.listdir(train_images_path))

filtered_labels_df = labels_df[labels_df['image_path'].isin(training_images)].copy()

filtered_labels_df.reset_index(drop=True, inplace=True)

print(f"Number of images in training folder: {len(training_images)}")
print(f"Number of rows in filtered_labels_df: {filtered_labels_df.shape[0]}")
filtered_labels_df.head()

Number of images in training folder: 17499
Number of rows in filtered_labels_df: 17499


,GalaxyID,Class1.1,Class1.2,Class1.3,Class2.1,Class2.2,Class3.1,Class3.2,Class4.1,Class4.2,...,Class10.1,Class10.2,Class10.3,Class11.1,Class11.2,Class11.3,Class11.4,Class11.5,Class11.6,image_path
0,100008,0.383147,0.616853,0.000000,0.000000,0.616853,0.038452,0.578401,0.418398,0.198455,...,0.279952,0.138445,0.000000,0.000000,0.092886,0.0,0.0,0.0,0.325512,100008.jpg
1,100023,0.327001,0.663777,0.009222,0.031178,0.632599,0.467370,0.165229,0.591328,0.041271,...,0.000000,0.131378,0.459950,0.000000,0.591328,0.0,0.0,0.0,0.000000,100023.jpg
2,100053,0.765717,0.177352,0.056931,0.000000,0.177352,0.000000,0.177352,0.000000,0.177352,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,100053.jpg
3,100078,0.693377,0.238564,0.068059,0.000000,0.238564,0.109493,0.129071,0.189098,0.049466,...,0.094549,0.000000,0.094549,0.189098,0.000000,0.0,0.0,0.0,0.000000,100078.jpg
4,100090,0.933839,0.000000,0.066161,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,100090.jpg


In [12]:

# Task 1: Smooth and rounded or disk viewed edge-on
conditions = [
    (filtered_labels_df["Class1.1"] >= 0.8),  # Task 1: Smooth and rounded
    (filtered_labels_df["Class2.1"] >= 0.8)   # Task 2: Disk viewed edge-on
]
choices = ["smooth", "disk_edge_on"]
filtered_labels_df["class"] = np.select(conditions, choices, default="uncertain")

# Task 2: Bar feature through the center of the galaxy
conditions = [
    (filtered_labels_df["Class3.1"] >= 0.5),  # Task 3: Bar feature
]
choices = ["bar_feature"]
filtered_labels_df["class"] = np.select(conditions, choices, default=filtered_labels_df["class"])

# Task 3: Spiral arm pattern check
conditions = [
    (filtered_labels_df["Class4.1"] >= 0.5),  # Task 4: Spiral arm pattern
]
choices = ["spiral_arm"]
filtered_labels_df["class"] = np.select(conditions, choices, default=filtered_labels_df["class"])

# Task 4: Bulge size comparison (no bulge, noticeable bulge, dominant bulge)
conditions = [
    (filtered_labels_df["Class5.1"] == 0),  # No bulge
    (filtered_labels_df["Class5.1"] == 1),  # Just noticeable bulge
    (filtered_labels_df["Class5.1"] == 2)   # Dominant bulge
]
choices = ["no_bulge", "noticeable_bulge", "dominant_bulge"]
filtered_labels_df["bulge"] = np.select(conditions, choices, default="uncertain")

# Task 5: Odd feature presence
conditions = [
    (filtered_labels_df["Class6.1"] >= 0.5),  # Task 6: Odd feature
]
choices = ["odd_feature"]
filtered_labels_df["odd_feature"] = np.select(conditions, choices, default="no_odd_feature")

# Task 6: Shape of galaxy (rounded, cigar-shaped, etc.)
conditions = [
    (filtered_labels_df["Class7.1"] >= 0.8),  # Task 7: Completely round
    (filtered_labels_df["Class7.1"] >= 0.5),  # Task 7: In between round
    (filtered_labels_df["Class7.1"] < 0.5)    # Task 7: Cigar-shaped
]
choices = ["completely_round", "in_between_round", "cigar_shaped"]
filtered_labels_df["shape"] = np.select(conditions, choices, default="uncertain")

# Task 7: Odd feature type (ring, disturbed, irregular, etc.)
conditions = [
    (filtered_labels_df["Class8.1"] == 1),  # Task 8: Ring
    (filtered_labels_df["Class8.1"] == 2),  # Task 8: Disturbed
    (filtered_labels_df["Class8.1"] == 3),  # Task 8: Irregular
    (filtered_labels_df["Class8.1"] == 4)   # Task 8: Other
]
choices = ["ring", "disturbed", "irregular", "other"]
filtered_labels_df["odd_feature_type"] = np.select(conditions, choices, default="uncertain")

# Task 8: Spiral arm tightness and number
conditions = [
    (filtered_labels_df["Class9.1"] == 1),  # Tight spiral arms
    (filtered_labels_df["Class9.1"] == 2),  # Medium spiral arms
    (filtered_labels_df["Class9.1"] == 3),  # Loose spiral arms
]
choices = ["tight_spiral", "medium_spiral", "loose_spiral"]
filtered_labels_df["spiral_arm_tightness"] = np.select(conditions, choices, default="uncertain")

# Task 9: Galaxy classification (elliptical, spiral, irregular)
conditions = [
    (filtered_labels_df["Class10.1"] >= 0.8),  # Task 9: Elliptical galaxy
    (filtered_labels_df["Class10.2"] >= 0.8),  # Task 9: Spiral galaxy
]
choices = ["elliptical", "spiral"]
filtered_labels_df["galaxy_class"] = np.select(conditions, choices, default="irregular")

# Task 10: Bulge at center (rounded, boxy, no bulge)
conditions = [
    (filtered_labels_df["Class11.1"] == 1),  # Rounded bulge
    (filtered_labels_df["Class11.1"] == 2),  # Boxy bulge
    (filtered_labels_df["Class11.1"] == 0)   # No bulge
]
choices = ["rounded_bulge", "boxy_bulge", "no_bulge"]
filtered_labels_df["bulge_type"] = np.select(conditions, choices, default="uncertain")

# --- Check Class Distribution and Take Top 5 Classes ---
# Check the distribution of the classes
class_counts = filtered_labels_df["class"].value_counts()

# Show the top 5 classes
top_5_classes = class_counts.head(5)
print("Top 5 classes by frequency:")
print(top_5_classes)

# Sample 8000 galaxies from the most frequent classes
sampled_labels_df = pd.DataFrame()

Top 5 classes by frequency:
class
uncertain       11503
spiral_arm       2917
smooth           2283
disk_edge_on      511
bar_feature       285
Name: count, dtype: int64


In [13]:
# --- Check Class Distribution and Take Top 5 Classes ---
# Check the distribution of the classes
class_counts = filtered_labels_df["class"].value_counts()

# Show the top 5 classes
top_5_classes = class_counts.head(5)
print("Top 5 classes by frequency:")
print(top_5_classes)

# Define the top 5 classes (including 'uncertain')
top_classes = top_5_classes.index.tolist()

# Create a new DataFrame with galaxy_id and class, filtered by the top 5 classes
filtered_labels_df1 = filtered_labels_df[filtered_labels_df["class"].isin(top_classes)][["GalaxyID", "class","image_path"]]

# Check the new DataFrame
print("Filtered DataFrame with Galaxy ID and Class for Top 5 Classes:")
print(filtered_labels_df.head())
print(filtered_labels_df.shape)

# Optionally, you can save the new DataFrame to a file, like a CSV
# filtered_labels_df.to_csv('filtered_top_5_classes.csv', index=False)


Top 5 classes by frequency:
class
uncertain       11503
spiral_arm       2917
smooth           2283
disk_edge_on      511
bar_feature       285
Name: count, dtype: int64
Filtered DataFrame with Galaxy ID and Class for Top 5 Classes:
   GalaxyID  Class1.1  Class1.2  Class1.3  Class2.1  Class2.2  Class3.1  \
0    100008  0.383147  0.616853  0.000000  0.000000  0.616853  0.038452   
1    100023  0.327001  0.663777  0.009222  0.031178  0.632599  0.467370   
2    100053  0.765717  0.177352  0.056931  0.000000  0.177352  0.000000   
3    100078  0.693377  0.238564  0.068059  0.000000  0.238564  0.109493   
4    100090  0.933839  0.000000  0.066161  0.000000  0.000000  0.000000   

   Class3.2  Class4.1  Class4.2  ...  Class11.6  image_path       class  \
0  0.578401  0.418398  0.198455  ...   0.325512  100008.jpg   uncertain   
1  0.165229  0.591328  0.041271  ...   0.000000  100023.jpg  spiral_arm   
2  0.177352  0.000000  0.177352  ...   0.000000  100053.jpg   uncertain   
3  0.129071  0.1

In [14]:
filtered_labels_df1.head()

,GalaxyID,class,image_path
0,100008,uncertain,100008.jpg
1,100023,spiral_arm,100023.jpg
2,100053,uncertain,100053.jpg
3,100078,uncertain,100078.jpg
4,100090,smooth,100090.jpg


In [15]:
# Define the target number of samples per class
target_samples = 500

# Create an empty DataFrame to store the balanced dataset
balanced_data = pd.DataFrame(columns=["GalaxyID", "class"])

# Loop through each class and sample the required number of images
for class_label in ["uncertain", "spiral_arm", "smooth", "disk_edge_on", "bar_feature"]:
    class_data = filtered_labels_df[filtered_labels_df["class"] == class_label][["GalaxyID", "class","image_path"]]
    if len(class_data) >= target_samples:
        # Sample exactly 500 images if possible
        sampled_data = class_data.sample(n=target_samples, random_state=42)
    else:
        # Take all available samples if less than 500
        sampled_data = class_data
        print(f"Warning: Class '{class_label}' has only {len(class_data)} samples. Using all available.")

    # Append the sampled data to the balanced DataFrame
    balanced_data = pd.concat([balanced_data, sampled_data], ignore_index=True)

# Reset the index of the balanced DataFrame
balanced_data.reset_index(drop=True, inplace=True)

# Check the structure of the new balanced DataFrame
print("Balanced DataFrame:")
print(balanced_data.head())

# Check the distribution in the new balanced DataFrame
print("\nClass distribution in the balanced dataset:")
print(balanced_data["class"].value_counts())


Balanced DataFrame:
  GalaxyID      class  image_path
0   337092  uncertain  337092.jpg
1   243130  uncertain  243130.jpg
2   228215  uncertain  228215.jpg
3   210378  uncertain  210378.jpg
4   251453  uncertain  251453.jpg

Class distribution in the balanced dataset:
class
uncertain       500
spiral_arm      500
smooth          500
disk_edge_on    500
bar_feature     285
Name: count, dtype: int64


In [ ]:
# Apply Label Encoding to the class column in balanced_data
label_encoder = LabelEncoder()
balanced_data['class_encoded'] = label_encoder.fit_transform(balanced_data['class'])

# Check the mapping of labels to encoded values
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print("Label Encoding Mapping:")
print(label_mapping)

# Check the updated balanced_data DataFrame
print("Balanced DataFrame with Encoded Classes:")
print(balanced_data.head())

In [17]:
import os
from PIL import Image


# Iterate through the GalaxyIDs in the balanced data
for galaxy_id in balanced_data["GalaxyID"].head():  # Adjust `.head()` or remove to check all
    image_file = f"{galaxy_id}.jpg"  # Assuming the images are in .jpeg format
    image_path = os.path.join(train_images_path, image_file)
    if os.path.exists(image_path):
        with Image.open(image_path) as img:
            print(f"GalaxyID: {galaxy_id}, Image: {image_file}, Size: {img.size}")
    else:
        print(f"Image for GalaxyID: {galaxy_id} not found.")


GalaxyID: 337092, Image: 337092.jpg, Size: (424, 424)
GalaxyID: 243130, Image: 243130.jpg, Size: (424, 424)
GalaxyID: 228215, Image: 228215.jpg, Size: (424, 424)
GalaxyID: 210378, Image: 210378.jpg, Size: (424, 424)
GalaxyID: 251453, Image: 251453.jpg, Size: (424, 424)


In [18]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

# Function to preprocess images
def preprocess_image(image_path):
    img = load_img(image_path, target_size=IMG_SIZE)
    img_array = img_to_array(img) / 255.0
    return img_array

image_paths = balanced_data['image_path'].apply(lambda x: os.path.join(train_images_path, x)).tolist()

image_data = np.array([preprocess_image(path) for path in image_paths])

print(f"Image data shape: {image_data.shape}")


Image data shape: (2285, 224, 224, 3)


In [22]:
# save preproceesed data
np.save(os.path.join(Data_path, "preprocessed_image_data.npy"), image_data)  # Save image data
print("Preprocessed data saved successfully!")

Preprocessed data saved successfully!


In [18]:
# File paths
image_data_file = os.path.join(Data_path, "preprocessed_image_data.npy")

image_data = np.load(image_data_file)

## **EDA**

### **Plot random galaxies**

### **Colour Histogram**

### **Correlation Matrix**

## **Data Splitting**

In [19]:
# Parameters for splitting
test_size = 0.2
val_size = 0.2
random_state = 42

# Split the data into training and testing sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(
    image_data, balanced_data, test_size=test_size, random_state=random_state
)

print(f"Training data shape: {X_train.shape}")
print(f"Training labels shape: {y_train.shape}")
print(f"Testing data shape: {X_test.shape}")
print(f"Testing labels shape: {y_test.shape}")

# Further split training data into training and validation sets (80% training, 20% validation)
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=val_size, random_state=random_state
)

print(f"Training data shape: {X_train.shape}")
print(f"Validation data shape: {X_val.shape}")
print(f"Training labels shape: {y_train.shape}")
print(f"Validation labels shape: {y_val.shape}")

Training data shape: (1828, 224, 224, 3)
Training labels shape: (1828, 3)
Testing data shape: (457, 224, 224, 3)
Testing labels shape: (457, 3)
Training data shape: (1462, 224, 224, 3)
Validation data shape: (366, 224, 224, 3)
Training labels shape: (1462, 3)
Validation labels shape: (366, 3)


In [25]:
# Define the data augmentation for training, and normalization for validation and test
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Create generators
train_generator = train_datagen.flow(
    X_train, y_train,        # Training data and labels
    batch_size=16,           # Number of samples per batch
    shuffle=True,            # Shuffle the data
    seed=random_state       # Ensure reproducibility
)

val_generator = val_datagen.flow(
    X_val, y_val,            # Validation data and labels
    batch_size=16,           # Number of samples per batch
    shuffle=False,           # Do not shuffle validation data
    seed=random_state       # Ensure reproducibility
)

test_generator = test_datagen.flow(
    X_test, y_test,          # Test data and labels
    batch_size=16,           # Number of samples per batch
    shuffle=False,           # Do not shuffle test data
    seed=random_state       # Ensure reproducibility
)

## **Model1 : VGG16**

### **Implementation**

**Load Model**

In [27]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
base_model = ResNet50(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(y_train.shape[1], activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [28]:
# Define callbacks
early_stopping = EarlyStopping(
    monitor='val_loss',  # Monitor validation loss
    patience=5,  # Stop after 5 epochs of no improvement
    verbose=1,  # Print a message when stopping
    restore_best_weights=True  # Restore the model with the best weights
)

model_checkpoint = ModelCheckpoint(
    'best_model.h5',  # Path to save the best model
    monitor='val_loss',  # Monitor validation loss to determine the best model
    save_best_only=True,  # Save only the best model based on validation loss
    verbose=1  # Print a message when saving the best model
)

**Define layers**

**Compile Model**

### **Training**